In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.nn import BCELoss
import torch.nn as nn


from importlib import reload

import vitmbt
import mbt
import ablation
import helpers
reload(mbt)
reload(ablation)
reload(helpers)
reload(vitmbt)
from helpers import ClassifierMetrics, display_tensor_as_rgb
from vitmbt import ViTMBT, train, val
from data import EmoDataset, new_collate_fn
import warnings
warnings.filterwarnings('ignore')

from constants import *

In [ ]:
DEVICE = "cuda"
PRETRAINED_CHKPT = "./pretrained_models/L_16-i21k-300ep-lr_0.001-aug_medium1-wd_0.1-do_0.1-sd_0.1--imagenet2012-steps_20k-lr_0.01-res_224.npz"
EPOCHS = 200
lr = 0.00001
BATCH_SZ = 2
LABELS = 8
vmbt = ViTMBT(1024, num_class=LABELS)
vmbt = nn.DataParallel(vmbt).cuda()

se = None
lines = None
train_ds = EmoDataset(f"{DATA_DIR}/Labels/train_labels_full_pruned.csv", nlines=lines, sole_emotion=se)
val_ds = EmoDataset(f"{DATA_DIR}/Labels/val_labels_full_pruned.csv", nlines=lines, sole_emotion=se)
test_ds = EmoDataset(f"{DATA_DIR}/Labels/test_labels_full_pruned.csv", nlines=lines, sole_emotion=se)

# train_metrics = ClassifierMetrics(task='multilabel', n_labels=LABELS, device=DEVICE)
# val_metrics = ClassifierMetrics(task='multilabel', n_labels=LABELS, device=DEVICE)

train_dl = DataLoader(train_ds, collate_fn=new_collate_fn, batch_size=BATCH_SZ, shuffle=True)
val_dl = DataLoader(val_ds, collate_fn=new_collate_fn, batch_size=BATCH_SZ, shuffle=False)
test_dl = DataLoader(test_ds, collate_fn=new_collate_fn, batch_size=BATCH_SZ, shuffle=False)

optimizer = torch.optim.AdamW(vmbt.parameters(), betas=(0.9, 0.999), lr=lr, weight_decay=1.0 / BATCH_SZ)
loss_func = BCELoss()
# loss_func = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(EPOCHS):
    train_metrics = train(vmbt, train_dl, optimizer, loss_fn=loss_func, cls_metrics=None)
    train_loss, train_f1, train_r, train_p, train_acc, train_bottleneck_tokens = train_metrics
    val_metrics = val(vmbt, val_dl, loss_fn=loss_func, cls_metrics=None)
    val_loss, val_f1, val_r, val_p, val_acc, val_bottleneck_tokens = val_metrics
    train_token = train_bottleneck_tokens[0][:, :256]
    val_token = val_bottleneck_tokens[0][:, :256]

    print(
        (f"Epoch {epoch + 1}: train_loss {train_loss:.5f}, val_loss {val_loss:.5f}\n"
            f"                   train_precision {train_p}, val_precision {val_p}\n"
            f"                   train_recall {train_r}, val_recall {val_r}\n"
            f"                   train_f1 {train_f1}, val_f1 {val_f1}"
            )
        )
    display_tensor_as_rgb(train_token, "Train bottleneck token") 
    